<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
!pip install tf-agents[reverb]

In [133]:
import yfinance as yfi
import pandas as pd
import asyncio
import tf_agents

In [134]:
def proccess_data(df):
    # Ensure the input is a DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    proccessed_data = pd.DataFrame()

    # Calculate the range
    proccessed_data['range'] = df['High'] - df['Low']

    stdev = proccessed_data['range'].std()

    # Calculate the percentage of the range that the closing price represents
    proccessed_data['close_percentage'] = (df['Close'] - df['Low']) / proccessed_data['range']

    proccessed_data['range'] = proccessed_data['range'] / stdev

    # Calculate the difference between the closing and opening prices
    proccessed_data['price_difference'] = (df['Close'] - df['Open']) / stdev


    proccessed_data.fillna(0, inplace=True)

    return proccessed_data

In [135]:
async def fetch_ticker_data(ticker: str, start_date: str, end_date: str, interval: str):
    try:
        data = yfi.download(ticker, start=start_date, end=end_date, interval=interval)
        proccessed_data = proccess_data(data)
        return proccessed_data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

async def fetch_all_tickers(ticker_list: list,start_date: str,end_date: str, interval: str, max_concurrency:int=50):
    tasks = [fetch_ticker_data(ticker,start_date=start_date,end_date=end_date,interval=interval) for ticker in ticker_list]
    return await asyncio.gather(*tasks[:max_concurrency])

In [136]:
start_date = '2023-01-01'
end_date = '2023-10-22'
interval='60m'

# List of forex tickers    # List of forex tickers
forex_tickers = ["EURUSD=X", "USDEUR=X","GBPUSD=X", 'USDGBP=x', "USDJPY=X","JPYUSD=X", "AUDUSD=X" , "USDAUD=X"]  # Example tickers

# Fetch data concurrently
results = await fetch_all_tickers(forex_tickers,start_date=start_date,end_date=end_date,interval=interval)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [137]:
class RotatingBuffer:
    def __init__(self, capacity, init_list = []):
        self.capacity = capacity
        self.arrays = init_list

    def append(self, array):
        self.arrays.append(array)
        if len(self.arrays) > self.capacity:
            self.arrays.pop(0)



    def get_buffer(self):
        return self.arrays.copy()


In [138]:
#data = data.values.tolist()

print(data[:window_size])

[[1.3640653230750164, 0.8180442595044448, 0.8680532966684117], [1.2404815724124605, 0.49968801996672213, -0.37229927391663487], [0.8672792857039281, 1.0, 0.0], [0.8680532966684117, 1.0, 0.37217027208922093], [1.3640653230750164, 0.18167202572347266, -0.7439535386962001], [0.8665052747394444, 0.14277207086496949, -0.4949800117872932], [1.732623543996647, 0.07132752587298041, -0.7421475131124049], [2.71626247802795, 0.49938259878419455, -0.37062225016025363], [2.0996337429893104, 0.4701400835586139, -0.3704932483328397], [1.9783720252202053, 0.06233698487219614, 0.0]]


In [176]:
from tf_agents.environments import py_environment, tf_py_environment
from tf_agents.specs import array_spec
import numpy as np
from tf_agents.trajectories import time_step as ts

window_size = 10
data = results[0].values.tolist()

class TradingEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')

    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(window_size,3), dtype=np.float32, name='observation')

    self._buffer = RotatingBuffer(window_size,data[:window_size])
    self._time_step = window_size
    self._state = self._buffer.get_buffer()
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = []
    self._episode_ended = False
    return ts.restart(np.array(self._state, dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    cur_data = data[self._time_step]

    self._buffer.append(cur_data)

    reward = cur_data[2] * (action-1)

    self._state = self._buffer.get_buffer()

    self._time_step += 1

    if self._time_step >= len(data)-1:
      self._episode_ended = True
      return ts.termination(np.array(self._state, dtype=np.float32), reward)
    else:
      return ts.transition(
          np.array(self._state, dtype=np.float32), reward=reward, discount=.9)

In [177]:
tf_env = TradingEnv()
tf_env = tf_py_environment.TFPyEnvironment(tf_env)
train_env = tf_env
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

# Test resetting the environment
time_step = tf_env.reset()
print("Initial state:", time_step.observation)

# Test stepping through the environment
for _ in range(5):
    action = 2
    next_time_step = tf_env.step(action)
    print(f"Observation: {next_time_step.observation}, Reward: {next_time_step.reward},action: {action}")

TimeStep Specs: TimeStep(
{'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(10, 3), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))})
Action Specs: BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32))
Initial state: tf.Tensor([], shape=(1, 0), dtype=int32)
Observation: [[[ 1.2404816   0.49968803 -0.37229928]
  [ 0.8672793   1.          0.        ]
  [ 0.8680533   1.          0.37217027]
  [ 1.3640653   0.18167202 -0.7439535 ]
  [ 0.86650527  0.14277206 -0.49498   ]
  [ 1.7326236   0.07132752 -0.7421475 ]
  [ 2.7162626   0.4993826  -0.37062225]
  [ 2.0996337   0.47014007 -0

In [180]:
from tf_agents.specs import tensor_spec
import tensorflow as tf
from tf_agents.networks import sequential
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common



def create_q_network(input_shape, num_actions):
    def dense_layer(num_units):
        return tf.keras.layers.Dense(
            num_units,
            activation=tf.keras.activations.relu,
            kernel_initializer=tf.keras.initializers.GlorotNormal())

    input_tensor_spec = tensor_spec.BoundedTensorSpec(
        shape=input_shape,
        dtype=tf.float32,
        minimum=0.0,
        maximum=1.0)

    network = sequential.Sequential([
        dense_layer(64),
        dense_layer(32),
        tf.keras.layers.Dense(num_actions, activation=None)
    ])

    return network

# Define the input shape and number of actions
input_shape = (10, 3)
num_actions = 3

network = create_q_network(input_shape, num_actions)

# Create the agent
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=network,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

ValueError: Expected q_network to emit a floating point tensor with inner dims (3,); but saw network output spec: TensorSpec(shape=(10, 3), dtype=tf.float32, name=None)
  In call to configurable 'DqnAgent' (<class 'tf_agents.agents.dqn.dqn_agent.DqnAgent'>)